# Projeto: sumarizador de documentos e extração de palavras-chave

Projeto em duplas.

Neste projeto vamos usar o que aprendemos sobre vetorização de documentos e similaridade para construir sumários de documentos. Os sumários serão construídos à partir das sentenças existentes no texto, como se estivéssemos usando uma caneta marcadora de texto para ressaltar as sentenças mais importantes. Este tipo de sumarização é conhecido como sumarização extrativa. 

## Sumarização

Você deve implementar duas técnicas de sumarização de documentos:

1. Clustering

Nesta técnica você deverá:

- Vetorizar as sentenças do documento. Teste as várias opções de vetorização que aprendemos (TF-IDF, Doc2Vec, LDA, etc).
- Agrupar as sentenças do documento em clusters usando K-Means.
- Escolher as sentenças mais próximas do centro do cluster, para cada cluster.
- Exibir estas sentenças na ordem em que se apresentaram no texto original.

2. TextRank

A ideia do TextRank é conectar todas as sentenças entre si através da sua similaridade (e.g. $1 - \text{distância cosseno}$) em uma *matriz de similaridade*. Nesta matriz a entrada $(i,j)$ representa a similaridade entre as sentenças $i$ e $j$. Sentenças mais informativas tendem a estar conectadas com várias outras sentenças do texto, e servirão como representantes dos assuntos que estão sendo discutidos nestas outras sentenças. Para determinar quais são as sentenças de maior conectividade vamos usar o algoritmo PageRank (https://en.wikipedia.org/wiki/PageRank), que já está implementado na biblioteca `networkx` em Python.

Veja o artigo https://www.analyticsvidhya.com/blog/2018/11/introduction-text-summarization-textrank-python/ para entender mais sobre essa técnica. Para conhecer os detalhes finos do algoritmo, veja o artigo original em https://web.eecs.umich.edu/~mihalcea/papers/mihalcea.emnlp04.pdf

Implemente o TextRank e use-o para determinar as sentenças mais importantes do documento. Apresente estas sentenças na ordem em que aparecem no documento original.

## Extração de palavras-chave

As mesmas técnicas de sumarização de documentos servem para extrair palavras-chave de documentos: basta considerar a similaridade entre embeddings de palavras ao invés de vetores de sentença!

Implemente também a extração de palavras-chave de documentos.

## Testando as implementações

Para testar suas implementações de sumarização você pode usar o dataset "CNN/Daily Mail" (https://github.com/abisee/cnn-dailymail). Cuidado: é um dataset bem grande, para testar seus desenvolvimentos é recomendável não rodar no corpus inteiro toda vez.

## Entregáveis

- O repositório com o código
- Um relatório completo: 
    - Introdução
        - Explicar o que é sumarização de texto, diferentes tipos, e fazer uma revisão bibliográfica pequena
        - Explicar os dois algoritmos
    - Métodos
        - Explicar o experimento: qual dataset, como vai medir desempenho, etc.
    - Resultados
        - Métricas automatizadas: ROUGE (https://pypi.org/project/rouge/)
        - Comparação qualitativa
    - Conclusão
    
## Rubrica

| Conceito | Definição |
|:--------:|:----------|
|    I     | Não entregou ou entregou nonsense |
|    D     | O relatório está incompleto ou com falhas, o código tem erros mas está aproximadamente correto, falta mais de uma implementação |
|    C     | O relatório está completo mas com escrita pobre, o código está meio bagunçado mas correto, falta uma implementação apenas |
|    B     | Bom relatório, à exceção do ROUGE. Boa implementação. |
|    A     | Implementou métrica ROUGE de desempenho. Aprimorou os métodos por conta própria, melhorando o desempenho. |

## 1 - Clustering

In [2]:
# vetorização
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import reuters
from nltk.corpus import brown
from pprint import pprint


In [5]:
docs = {}
for fileid in reuters.fileids():
    docs[fileid] = reuters.sents(fileid)

for k, v in docs.items():
    docs[k] = []
    sent = ''
    for s in v:
        sent = ' '.join(s)
        docs[k].append(sent)   

In [7]:
doc_sents = {}
all_sents = []
num_all_sents = 0

for doc_id, doc in docs.items():
    all_sents += doc
    num_sents = len(doc)
    doc_sents[doc_id] = list(range(num_all_sents, num_all_sents + num_sents))
    num_all_sents+=num_sents


In [21]:
#retirar stopwords

In [8]:
vectorizer = TfidfVectorizer(analyzer=lambda x: x)
vectors = vectorizer.fit_transform(docs)

In [9]:
print(vectorizer.get_feature_names()[:20])

['/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'a', 'e', 'g', 'i', 'n', 'r', 's', 't']


In [10]:
import numpy as np
# similaridade entre sentenças
v0 = vectors[0]
v1 = vectors[2]
print(v0.dot(v1.transpose()))

  (0, 0)	0.8746068145112749


In [11]:
print(vectors.shape)
print(num_all_sents)

(10788, 19)
54716


In [30]:
vectors[0:31].shape

(31, 19)

## Clustering TF-IDF

In [12]:
from sklearn.cluster import MiniBatchKMeans
from sklearn.metrics import pairwise_distances_argmin_min
import pandas as pd

for doc, sent_indexes in doc_sents.items():
    num_sents = len(sent_indexes)
    
    first_index = doc_sents[doc][0]
    last_index = doc_sents[doc][-1] + 1
    
    if num_sents >= 2:
        kmeans = MiniBatchKMeans(n_clusters= 2, random_state=42)
        result = kmeans.fit_transform(vectors[first_index:last_index])

        closest, dis = pairwise_distances_argmin_min(kmeans.cluster_centers_, vectors[first_index:last_index])
        
        closest.sort()
        
        texts_centroids=[]
        for e in closest:
            sent = all_sents[e+first_index]

            texts_centroids.append(sent)
    
        print(f'doc: {doc} ---------------------------------')    
        print(texts_centroids)
#expandir

doc: test/14826 ---------------------------------
['Threat against Japan because it serves as a warning to us ," said a senior Taiwanese trade official who asked not to be named .', "Deputy U . S . Trade Representative Michael Smith and Makoto Kuroda , Japan ' s deputy minister of International Trade and Industry ( MITI ), are due to meet in Washington this week in an effort to end the dispute ."]
doc: test/14828 ---------------------------------
['The paper blamed the waste on inadequate storage and bad preservation methods .', 'The paper gave no further details .']
doc: test/14829 ---------------------------------
['MITI is planning to work out a revised energy supply / demand outlook through deliberations of committee meetings of the Agency of Natural Resources and Energy , the officials said .', 'They said MITI will also review the breakdown of energy supply sources , including oil , nuclear , coal and natural gas .']
doc: test/14832 ---------------------------------
["THAI TRADE D

ValueError: Found array with 0 sample(s) (shape=(0, 19)) while a minimum of 1 is required.

## Clustering usando cbow

In [14]:
import gensim
with open('sentences.txt', 'w', encoding='utf8') as file:
    for sentence in all_sents:
        file.write(f'{sentence}\n')

In [15]:
%%time
model_cbow = gensim.models.Word2Vec(
    corpus_file='sentences.txt',
    window=5,
    size=200,
    seed=42,
    iter=100,
    workers=12,
)

CPU times: user 8min 27s, sys: 14.9 s, total: 8min 42s
Wall time: 1min 44s


In [16]:
def cbow(model, sent):
    vec = np.zeros(model.wv.vector_size)
    for word in sent:
        if word in model:
            vec += model.wv.get_vector(word)
            
    norm = np.linalg.norm(vec)
    if norm > np.finfo(float).eps:
        vec /= norm
    return vec

In [18]:
vecs_cbow = [cbow(model_cbow, sent) for sent in all_sents]

/Users/elisamalzoni/nlp-env/lib/python3.7/site-packages/ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
  after removing the cwd from sys.path.


In [ ]:
for doc, sent_indexes in doc_sents.items():
    num_sents = len(sent_indexes)
    
    first_index = doc_sents[doc][0]
    last_index = doc_sents[doc][-1] + 1
    
    if num_sents >= 2:
        kmeans_cbow = MiniBatchKMeans(n_clusters= 2, random_state=42)
        result = kmeans_cbow.fit_transform(vecs_cbow[first_index:last_index])

        closest, dis = pairwise_distances_argmin_min(kmeans_cbow.cluster_centers_, vecs_cbow[first_index:last_index])
        closest.sort()
        texts_centroids=[]
        
        for e in closest:
            sent = all_sents[e+first_index]

            texts_centroids.append(sent)
    
        print(f'doc: {doc} ---------------------------------')    
        print(texts_centroids)
#expandir

doc: test/14826 ---------------------------------
["The U . S . Has said it will impose 300 mln dlrs of tariffs on imports of Japanese electronics goods on April 17 , in retaliation for Japan ' s alleged failure to stick to a pact not to sell semiconductors on world markets at below cost .", 'In Hong Kong , where newspapers have alleged Japan has been selling below - cost semiconductors , some electronics manufacturers share that view .']
doc: test/14828 ---------------------------------
["CHINA DAILY SAYS VERMIN EAT 7 - 12 PCT GRAIN STOCKS A survey of 19 provinces and seven cities showed vermin consume between seven and 12 pct of China ' s grain stocks , the China Daily said .", 'It said the government had launched a national programme to reduce waste , calling for improved technology in storage and preservation , and greater production of additives .']
doc: test/14829 ---------------------------------
['MITI is planning to work out a revised energy supply / demand outlook through del

## 2 - TextRank

Utilizando o tutorial - https://www.analyticsvidhya.com/blog/2018/11/introduction-text-summarization-textrank-python/

In [ ]:
import numpy as np
import pandas as pd
import nltk
nltk.download('punkt') # one time execution
import re

In [ ]:
df = pd.read_csv('tennis_articles_v4.csv')

In [ ]:
df.head()

In [ ]:
df['article_text'][0]

In [ ]:
from nltk.tokenize import sent_tokenize
from pprint import pprint
sentences = []
for s in df['article_text']:
    sentences.append(sent_tokenize(s))
# print(sentences)
doc_sents = {}
all_sents = []
num_all_sents = 0

for index, doc in enumerate(sentences):
    all_sents += doc
    num_sents = len(doc)
    doc_sents[index] = list(range(num_all_sents, num_all_sents + num_sents))
    num_all_sents+=num_sents                        

pprint(doc_sents)
sentences = [y for x in sentences for y in x] # flatten list
# print(sentences)

In [ ]:
# !wget http://nlp.stanford.edu/data/glove.6B.zip
# !unzip glove*.zip

In [ ]:
# Extract word vectors
word_embeddings = {}
f = open('glove.6B.100d.txt', encoding='utf-8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    word_embeddings[word] = coefs
f.close()

In [ ]:
# remove punctuations, numbers and special characters
clean_sentences = pd.Series(sentences).str.replace("[^a-zA-Z]", " ")

# make alphabets lowercase
clean_sentences = [s.lower() for s in clean_sentences]

In [ ]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

In [ ]:
# function to remove stopwords
def remove_stopwords(sen):
    sen_new = " ".join([i for i in sen if i not in stop_words])
    return sen_new

In [ ]:
# remove stopwords from the sentences
clean_sentences = [remove_stopwords(r.split()) for r in clean_sentences]

In [ ]:
# Extract word vectors
word_embeddings = {}
f = open('glove.6B.100d.txt', encoding='utf-8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    word_embeddings[word] = coefs
f.close()

In [ ]:
sentence_vectors = []
for i in clean_sentences:
    if len(i) != 0:
        v = sum([word_embeddings.get(w, np.zeros((100,))) for w in i.split()])/(len(i.split())+0.001)
    else:
        v = np.zeros((100,))
    sentence_vectors.append(v)

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
import networkx as nx

In [ ]:
for doc, sent_indexes in doc_sents.items():
    num_sents = len(sent_indexes)
    # similarity matrix
    sim_mat = np.zeros([num_sents, num_sents])
    for i, s1 in enumerate(sent_indexes):
        for j, s2 in enumerate(sent_indexes):
            if i != j:
                #indice sentece_vector aqui deveria 
                sim_mat[i][j] = cosine_similarity(sentence_vectors[s1].reshape(1,100), sentence_vectors[s2].reshape(1,100))[0,0]
    
    nx_graph = nx.from_numpy_array(sim_mat)
    scores = nx.pagerank(nx_graph)
    
    first_index = doc_sents[doc][0]
    last_index = doc_sents[doc][-1] + 1
    
    ranked_sentences = sorted(((scores[i],s) for i,s in enumerate(sentences[first_index:last_index])), reverse=True)
    print(f'doc: {doc} \n')
    for i in range(3):
        print(ranked_sentences[i][1])
    print('*' * 30)
    print(sentences[first_index:last_index])
    print('-' * 50)

In [ ]:

#o grafo é fully connected?
nx_graph = nx.from_numpy_array(sim_mat)
scores = nx.pagerank(nx_graph)

In [ ]:
ranked_sentences = sorted(((scores[i],s) for i,s in enumerate(sentences)), reverse=True)

In [ ]:
# Extract top 10 sentences as the summary
for i in range(10):
    print(ranked_sentences[i][1])